## Pendulum Downward Using Lyapunov and Upward using LQR and Lyapunov

In [1]:
import scipy
import control
import numpy as np
import sympy as sp
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

from pydrake.all import (LinearQuadraticRegulator, MathematicalProgram, Variables,
                         Solve, RealContinuousLyapunovEquation)

import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import (Jacobian, MathematicalProgram, Polynomial,
                         RegionOfAttraction, RegionOfAttractionOptions, Solve,
                         SymbolicVectorSystem, ToLatex, Variable, Variables,
                         plot_sublevelset_expression)
from pydrake.examples.pendulum import PendulumParams
from pydrake.examples.van_der_pol import VanDerPolOscillator
from pydrake.symbolic import Polynomial

from underactuated import plot_2d_phase_portrait, running_as_notebook

from pydrake.all import (AddMultibodyPlantSceneGraph, ControllabilityMatrix,
                         DiagramBuilder, Linearize, LinearQuadraticRegulator,
                         MeshcatVisualizerCpp, Parser, Saturation, SceneGraph,
                         Simulator, StartMeshcat, WrapToSystem)

if running_as_notebook:
    mpld3.enable_notebook()

import math

In [2]:
from pydrake.examples.pendulum import PendulumParams, PendulumPlant

p = PendulumParams()

b = p.damping()
m = p.mass()
length = p.length()
g = p.gravity()
grav = p.gravity()

## Simple Pendulum Example

Take the Lyapunov function at the vertical down position and verify that it will grow as the alternations converge. 

In [4]:
# explicit dynamics
plant = PendulumPlant()
context = plant.CreateDefaultContext()
plant.get_input_port().FixValue(context, [0])
context.SetContinuousState([0, 0])

linearized_plant = Linearize(plant, context)

A = linearized_plant.A()
Q = np.eye(2)
P = RealContinuousLyapunovEquation(A, Q)
X1 = control.lyap(A.T, Q)

In [5]:
#doing the van der pol oscillator implicitly

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

negative_infinity = float('-inf')

def find_implicit_lambdas(x, xd, V, Vdot, g):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(V.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
    
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + V.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
  
    prog.AddSosConstraint(-Vdot + lambda_*(V - 1) + lambda_g.dot(g))
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(1e-5))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(1e-5), np.array(lambda_g_results)

def find_implicit_V(x, xd, lambda_, lambda_g, g, V_degree):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
    
    y = np.hstack([x, xd])
        
    V, Q = prog.NewSosPolynomial(Variables(x), V_degree)
    Vdot = V.Jacobian(x).dot(xd)
    
    prog.AddSosConstraint(-Vdot + lambda_*(V - 1) + lambda_g.dot(g))
    
    prog.AddLinearCost(np.trace(Q))    
    #prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:0, xd[0]: 0, xd[1]: 0}) == 0)
    prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:0}) == 0)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("V solution status: " + str(k))
    
    assert result.is_success()
        
    return result.GetSolution(V), result.GetSolution(Q)


def problem_solver_implicit(S, V_degree):
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(2, "x")
    xd = prog.NewIndeterminates(2, "xd")
        
    g0 = xd[0] - x[1]
    g1 = m*(length**2)*xd[1] + b*x[1] + m*grav*length*x[0]
    
    g = np.array([g0, g1])
    
    V = x.T@P@x
    Vdot = V.Jacobian(x).dot(xd)
    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(200):
        lambda_, lambda_g = find_implicit_lambdas(x, xd, V, Vdot, g_poly)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
#         print(alpha)
        
        V, Q = find_implicit_V(x, xd, lambda_, lambda_g, g_poly, V_degree)
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
    return V, Q

V_degree = 2
V, Q = problem_solver_implicit(P, V_degree)

$ V(x)=(0.050968400x_{0} x_{1} + 25.785193680x_{0}^{2} + 1.313710499x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} - 0.000000000x_{1} + 0.058812414x_{0} x_{1} + 25.148088222x_{0}^{2} + 1.279074892x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} - 0.000000000x_{1} + 0.063062259x_{0} x_{1} + 24.528546646x_{0}^{2} + 1.245323622x_{1}^{2})$

...

$ V(x)=(0.000000000 + 0.000000000x_{0} + 0.000000000x_{1} + 0.000013287x_{0} x_{1} + 0.023648981x_{0}^{2} + 0.001202509x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} - 0.000000000x_{1} + 0.000010087x_{0} x_{1} + 0.021748206x_{0}^{2} + 0.001105852x_{1}^{2})$

Lambda solution status: 1


KeyboardInterrupt: 

In [57]:
#THIS CODE WORKS
from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

def findLambda(x, V, Vdot, count):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    
    #V = Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression()
    
    lambda_ = prog.NewSosPolynomial(Variables(x), 2)[0].ToExpression()
        
    prog.AddSosConstraint(-Vdot - lambda_*(1 - V))
    prog.AddSosConstraint(lambda_)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
        
    print('Status of first solver on step ' + str(count) + ': ' + str(k))
    assert result.is_success()
    
    return result.GetSolution(lambda_)


def findV(x, lambda_, f, count):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    
    lambda_ = Polynomial(lambda_).RemoveTermsWithSmallCoefficients(1e-2).ToExpression()
        
    temp, Q = prog.NewSosPolynomial(Variables(x), 2)
    V = temp.ToExpression()
            
    prog.AddLinearConstraint(V.Substitute({x[0]: 0, x[1]:0}) == 0)
    
    Vdot = V.Jacobian(x).dot(f)
    
    prog.AddSosConstraint(-Vdot - lambda_*(1 - V))
    
#     prog.AddSosConstraint(np.trace(Q))
    
    A = Q[0, 0]
    B = 2*Q[1, 0]
    C = Q[1, 1]
    
    prog.AddCost(A)
    prog.AddCost(C)
    prog.AddCost(-B)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
        
    k = result.get_solver_details().solution_status
                
    print('Status of second solver on step ' + str(count) + ': ' + str(k))
    assert result.is_success()
    
    return result.GetSolution(V), result.GetSolution(Q)

def setup_solver():  
    count = 0
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(2, "x")
    
    f0 = x[1]
    f1 = (-b/(m*length**2))*x[1] - (grav/length)*(x[0])
    
    f = np.array([f0, f1])
    
    V = x.T@P@x
    
    display(Markdown("$V(x) = " + ToLatex(
    Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
    #Vdot = x.T@P@A@x + x.T@A.T@P@x
    Vdot = V.Jacobian(x).dot(f)
    
    for i in range(20): 
        lambda_ = findLambda(x, V, Vdot, count)
        
#         display(Markdown("$\lambda (x) = " + ToLatex(
#         Polynomial(lambda_).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
        V, Q = findV(x, lambda_, f, count)
        
        display(Markdown("$V(x) = " + ToLatex(
        Polynomial(V).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
        Vdot = V.Jacobian(x).dot(f)
        
        count += 1
    return V, Q
    
    
V, Q = setup_solver()

$V(x) = (0.050968x_{0} x_{1} + 25.785194x_{0}^{2} + 1.313710x_{1}^{2})$

Status of first solver on step 0: 1
Status of second solver on step 0: 1


$V(x) = (0.004926x_{0} x_{1} + 3.587715x_{0}^{2} + 0.182810x_{1}^{2})$

Status of first solver on step 1: 1
Status of second solver on step 1: 1


$V(x) = (0.002776x_{0} x_{1} + 2.291521x_{0}^{2} + 0.116767x_{1}^{2})$

...

$V(x) = (0.297182x_{0}^{2} + 0.015147x_{1}^{2})$

Status of first solver on step 18: 1
Status of second solver on step 18: 1


$V(x) = (0.266463x_{0}^{2} + 0.013581x_{1}^{2})$

Status of first solver on step 19: 1
Status of second solver on step 19: 1


$V(x) = 0$

### Pendulum Upright Example
Goal is to stabalize the pendulum in the upward position and certify the region of stability using Lyapunov analysis after we have found the controller.  

For this we first need to do a change or coordinates: $\tilde{\theta} = \theta - \pi$ 

The dynamics now become: 
$ml^2\ddot{\tilde{\theta}} + b\dot{\tilde{\theta}} - mgl\sin{\tilde{\theta}} = 0$

Here $\theta$ is the angle with the vertical when in the downward position as defined by default in Drake

In [5]:
p = PendulumParams()

# Add the Linearization
A = np.array([[0, 1], [p.mass()*p.gravity()*p.length(), -p.damping()]])
B = np.array([[0], [1]])
E = np.array([[1, 0], [0, p.mass()*p.length()*p.length()]])

# Add LQR Costs
Q = np.diag([10, 1])
R = [1]

# matrix needed for algorithm
S = np.array([[0], [0]])

In [6]:
# solve the generalized care
X, L, G = control.care(A, B, Q, R, S, E)

In [7]:
# G = R^-1B^TXE is the gain matrix to become A - BG for closed loop
G

array([[10.74101105,  2.42596626]])

In [8]:
G_casted = np.insert(G, 1, 0)
G_casted

array([10.74101105,  0.        ,  2.42596626])

To do Lyapunov analysis: close the loop and solve the linear Lyapunov equation. Then, use the dynamics to find the ROA

In [9]:
A_closed = A - B@G

Q_ly = np.eye(2)
P = control.lyap(A_closed.T, Q_ly, E = E.T)
candidate_matrix = E.T@P@E

u = -Gx. Remember to add this to the dynamics equations in the SOS program

In [12]:
# Use this as a starting candidate for ROA study

#doing the van der pol oscillator implicitly

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

negative_infinity = float('-inf')

def find_implicit_lambdas(x, xd, V, Vdot, g):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(V.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
    
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + V.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
  
    prog.AddSosConstraint(-Vdot + lambda_*(V - 1) + lambda_g.dot(g))
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(1e-5))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(1e-5), np.array(lambda_g_results)

def find_implicit_V(x, xd, lambda_, lambda_g, g, V_degree):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
    
    y = np.hstack([x, xd])
        
    V, Q = prog.NewSosPolynomial(Variables(x), V_degree)
    Vdot = V.Jacobian(x).dot(xd)
    
    prog.AddSosConstraint(-Vdot + lambda_*(V - 1) + lambda_g.dot(g))
    
    prog.AddLinearCost(np.trace(Q))    
    #prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:0, xd[0]: 0, xd[1]: 0}) == 0)
    prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:0}) == 0)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("V solution status: " + str(k))
    
    assert result.is_success()
        
    return result.GetSolution(V), result.GetSolution(Q)


def problem_solver_implicit(S, V_degree, G):
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(2, "x")
    xd = prog.NewIndeterminates(2, "xd")
    
    u = -G@x
    sinx = x[0]
    
    g0 = xd[0] - x[1]
    g1 = p.mass()*(p.length()**2)*xd[1] + p.damping()*x[1] - p.mass()*p.gravity()*p.length()*sinx - u[0]
    
    g = np.array([g0, g1])
    
    V = x.T@S@x
    Vdot = V.Jacobian(x).dot(xd)
    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(200):
        lambda_, lambda_g = find_implicit_lambdas(x, xd, V, Vdot, g_poly)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
#         print(alpha)
        
        V, Q = find_implicit_V(x, xd, lambda_, lambda_g, g_poly, V_degree)
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
    return V, Q

V_degree = 2
V, Q = problem_solver_implicit(candidate_matrix, V_degree, G)

$ V(x)=(0.042837479x_{0} x_{1} + 1.421101750x_{0}^{2} + 0.051605869x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} - 0.000000000x_{1} + 0.047357625x_{0} x_{1} + 1.205675635x_{0}^{2} + 0.043871794x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000000 - 0.000000000x_{0} + 0.000000000x_{1} + 0.048533584x_{0} x_{1} + 1.019652540x_{0}^{2} + 0.036714577x_{1}^{2})$

...

$ V(x)=(0.000000001 - 0.000000000x_{0} + 0.000000000x_{1} - 0.000000002x_{0} x_{1} + 0.000000003x_{0}^{2} + 0.000000002x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000001 - 0.000000000x_{0} - 0.000000000x_{1} - 0.000000002x_{0} x_{1} + 0.000000003x_{0}^{2} + 0.000000002x_{1}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.000000003 + 0.000000000x_{0} + 0.000000000x_{1} - 0.000000003x_{0} x_{1} + 0.000000005x_{0}^{2} + 0.000000004x_{1}^{2})$

In [13]:
# now the states become sin(theta), theta_dot
# the controller K gets multiplies by K and the new state
# input this U in the equations of motion
# now linearize
# feed this linearization into the 

In [14]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols as dynamicsymbols
from sympy import sin as symsin
from sympy import cos as symcos

In [15]:
theta, s, c = dynamicsymbols('theta s c')
t = sp.symbols('t')
theta_dot_initial = dynamicsymbols('theta_dot_initial')

theta_dot = sp.diff(theta, t)

m = p.mass()
grav = p.gravity()
length = p.length()
b = p.damping()

state = np.array([theta, theta_dot_initial])

V = state.T@candidate_matrix@state
print(type(V))
V = sp.simplify(V)

print(V)

subs_dict = {theta**2: (2-2*c), theta: s}
V = V.subs(subs_dict)

subs_dict_2 = {theta_dot_initial: theta_dot}
V = V.subs(subs_dict_2)

<class 'sympy.core.add.Add'>
1.42110175013269*theta(t)**2 + 0.0428374788479707*theta(t)*theta_dot_initial(t) + 0.0516058694571062*theta_dot_initial(t)**2


In [16]:
V

-2.84220350026538*c(t) + 0.0428374788479707*s(t)*Derivative(theta(t), t) + 0.0516058694571062*Derivative(theta(t), t)**2 + 2.84220350026538

In [17]:
candidate_matrix_new = np.array([[0.0516, 0, 0.021, 0], [0, 0, 0, -1.42], [0.021, 0, 0, 0], [0, -1.42, 0, 2.8422]])

print(np.linalg.eigvals(candidate_matrix_new))
print(np.linalg.eigvals(candidate_matrix))

[ 0.0590662  -0.0074662  -0.58786123  3.43006123]
[1.42143665 0.05127097]


In [18]:
# Use this as a starting candidate for ROA study

#doing the van der pol oscillator implicitly

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

negative_infinity = float('-inf')
coeff = 1e-5

def find_implicit_lambdas(x, xd, V, Vdot, g):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(V.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
    
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + V.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
    
    s_deg = max(l_deg + V.TotalDegree() - 2, 0)
    lambda_s = prog.NewFreePolynomial(Variables(y), s_deg)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)
  
    prog.AddSosConstraint(-Vdot + lambda_*(V - 0.001) + lambda_g.dot(g) - lambda_s*trig)
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(coeff))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(coeff), np.array(lambda_g_results), \
result.GetSolution(lambda_s).RemoveTermsWithSmallCoefficients(coeff)

def find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g, V_degree):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
            
    V, Q = prog.NewSosPolynomial(Variables(x), V_degree)
    Vdot = V.Jacobian(x).dot(xd)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)
    prog.AddSosConstraint(-Vdot + lambda_*(V - 0.001) + lambda_g.dot(g) - lambda_s*trig)
    
    mgl = p.mass()*p.length()*p.gravity()
    
    prog.AddLinearCost(np.trace(Q))    
    prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 1, x[1]:1, x[2]:0}) == 0)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("V solution status: " + str(k))
    
    assert result.is_success()
        
    return result.GetSolution(V), result.GetSolution(Q)


def problem_solver_implicit(S, V_degree, G):
    prog = MathematicalProgram()
    s = prog.NewIndeterminates(1, "s")
    c = prog.NewIndeterminates(1, "c")
    theta_dot = prog.NewIndeterminates(1, "theta_dot")
    
    xd = prog.NewIndeterminates(3, 'xd')
    
    state_V = np.hstack([theta_dot, c, s, [1]])
    
    x = np.hstack([s, c, theta_dot])
    
    u = -G@x
        
    g0 = xd[0] - c[0]*theta_dot[0]
    g1 = xd[1] + s[0]*theta_dot[0]
    g2 = p.mass()*(p.length()**2)*xd[2] + p.damping()*theta_dot[0] - p.mass()*p.gravity()*p.length()*s[0] - u
    
    g = np.array([g0, g1, g2])
    
    V = state_V.T@S@state_V
    Vdot = V.Jacobian(x).dot(xd)
    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(100):
        lambda_, lambda_g, lambda_s = find_implicit_lambdas(x, xd, V, Vdot, g_poly)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_s.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
        
        V, Q = find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g_poly, V_degree)
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
    return V, Q

V_degree = 2
V, Q = problem_solver_implicit(candidate_matrix_new, V_degree, G_casted)

$ V(x)=(2.842200000 - 2.840000000c_{0} + 0.042000000s_{0} theta_dot_{0} + 0.051600000theta_dot_{0}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(6.631793360 - 3.387416523s_{0} - 9.876103091c_{0} - 0.251230409theta_dot_{0} + 2.522214577s_{0} c_{0} + 0.063035176s_{0} theta_dot_{0} + 0.188193977c_{0} theta_dot_{0} + 0.432592411s_{0}^{2} + 3.676919265c_{0}^{2} + 0.016355774theta_dot_{0}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(4.827050655 - 3.249087412s_{0} - 6.404994502c_{0} - 0.781835261theta_dot_{0} + 2.153058722s_{0} c_{0} + 0.244560905s_{0} theta_dot_{0} + 0.537272752c_{0} theta_dot_{0} + 0.548011083s_{0}^{2} + 2.125961453c_{0}^{2} + 0.099469906theta_dot_{0}^{2})$

...

$ V(x)=(0.011656410 - 0.011656652s_{0} - 0.011655968c_{0} + 0.000000012theta_dot_{0} + 0.005828092s_{0} c_{0} + 0.000000002s_{0} theta_dot_{0} - 0.000000014c_{0} theta_dot_{0} + 0.002914230s_{0}^{2} + 0.002913887c_{0}^{2} + 0.000000066theta_dot_{0}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.011656447 - 0.011656257s_{0} - 0.011656472c_{0} - 0.000000002theta_dot_{0} + 0.005828128s_{0} c_{0} + 0.000000006s_{0} theta_dot_{0} - 0.000000004c_{0} theta_dot_{0} + 0.002914023s_{0}^{2} + 0.002914131c_{0}^{2} + 0.000000061theta_dot_{0}^{2})$

Lambda solution status: 1
V solution status: 1


$ V(x)=(0.011656402 - 0.011656106s_{0} - 0.011656405c_{0} + 0.000000084theta_dot_{0} + 0.005828039s_{0} c_{0} - 0.000000043s_{0} theta_dot_{0} - 0.000000041c_{0} theta_dot_{0} + 0.002913960s_{0}^{2} + 0.002914110c_{0}^{2} + 0.000000048theta_dot_{0}^{2})$

In [30]:
# finding the control input itself 

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

negative_infinity = float('-inf')
coeff = 1e-5

def find_implicit_lambdas(x, xd, V, Vdot, g):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
    
    
    print(g[2])
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(V.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
        
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + V.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
    
    s_deg = max(l_deg + V.TotalDegree() - 2, 0)
    lambda_s = prog.NewFreePolynomial(Variables(y), s_deg)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)
  
    prog.AddSosConstraint(-Vdot + lambda_*(V - 0.001) + lambda_g.dot(g) - lambda_s*trig)
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(coeff))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(coeff), np.array(lambda_g_results), \
result.GetSolution(lambda_s).RemoveTermsWithSmallCoefficients(coeff)

def find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g, V_degree):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
            
    V, Q = prog.NewSosPolynomial(Variables(x), V_degree)
    Vdot = V.Jacobian(x).dot(xd)
    
    # the u business
    u = prog.NewFreePolynomial(Variables(x), 1)
    
    g2 = Polynomial(p.mass()*(p.length()**2)*xd[2] + p.damping()*x[2] \
                    - p.mass()*p.gravity()*p.length()*x[0]) - u
    g[2] = g2
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)
    prog.AddSosConstraint(-Vdot + lambda_*(V - 0.001) + lambda_g.dot(g) - lambda_s*trig)
    
    mgl = p.mass()*p.length()*p.gravity()
    
    prog.AddLinearCost(np.trace(Q))    
    prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 1, x[1]:1, x[2]:0}) == 0)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("V solution status: " + str(k))
    
    assert result.is_success()
        
    return result.GetSolution(V), result.GetSolution(Q), result.GetSolution(u)


def problem_solver_implicit(S, V_degree, G):
    prog = MathematicalProgram()
    s = prog.NewIndeterminates(1, "s")
    c = prog.NewIndeterminates(1, "c")
    theta_dot = prog.NewIndeterminates(1, "theta_dot")
    
    xd = prog.NewIndeterminates(3, 'xd')
    
    state_V = np.hstack([theta_dot, c, s, [1]])
    
    x = np.hstack([s, c, theta_dot])
    
    u = -G@x
        
    g0 = xd[0] - x[1]*x[2]
    g1 = xd[1] + x[0]*x[2]
    g2 = p.mass()*(p.length()**2)*xd[2] + p.damping()*x[2] - p.mass()*p.gravity()*p.length()*x[0] - u
    
    g = np.array([g0, g1, g2])
    
    V = state_V.T@S@state_V
    Vdot = V.Jacobian(x).dot(xd)
    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(50):
        lambda_, lambda_g, lambda_s = find_implicit_lambdas(x, xd, V, Vdot, g_poly)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_s.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
        
        V, Q, u = find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g_poly, V_degree)
        
        g2 = Polynomial(p.mass()*(p.length()**2)*xd[2] + p.damping()*x[2] \
                    - p.mass()*p.gravity()*p.length()*x[0]) - u
        g_poly[2] = g2
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
        display(Markdown("$ u(x)="+ToLatex(u.ToExpression(), 9)+"$"))
        
    return V, Q, u

V_degree = 2
V, Q, u = problem_solver_implicit(candidate_matrix_new, V_degree, G_casted)

$ V(x)=(2.842200000 - 2.840000000c_{0} + 0.042000000s_{0} theta_dot_{0} + 0.051600000theta_dot_{0}^{2})$

0.25*xd(2) + 2.5259662559187128*theta_dot(0) + 5.8360110520799973*s(0)
Lambda solution status: 1
V solution status: 1


$ V(x)=(4.115838798 - 3.043749965s_{0} - 5.187773415c_{0} + 0.646126512theta_dot_{0} + 1.912004561s_{0} c_{0} - 0.208852596s_{0} theta_dot_{0} - 0.437262844c_{0} theta_dot_{0} + 0.565844074s_{0}^{2} + 1.637835948c_{0}^{2} + 0.097994607theta_dot_{0}^{2})$

$ u(x)=(4.265839193 - 7.895268924s_{0} - 2.631571825c_{0} - 1.633031763theta_dot_{0})$

-4.2658391926383619*1 + 0.25*xd(2) + 1.7330317632312897*theta_dot(0) + 2.6315718254680505*c(0) + 2.9902689243741225*s(0)
Lambda solution status: 1
V solution status: 1


...

$ V(x)=(0.011617804 - 0.011609173s_{0} - 0.011609206c_{0} - 0.000000244theta_dot_{0} + 0.005799513s_{0} c_{0} + 0.000000136s_{0} theta_dot_{0} + 0.000000141c_{0} theta_dot_{0} + 0.002900526s_{0}^{2} + 0.002900543c_{0}^{2} + 0.000004850theta_dot_{0}^{2})$

$ u(x)=(-0.597645536 - 4.830332561s_{0} + 0.064423930c_{0} - 0.301417902theta_dot_{0})$

0.59764553600553827*1 + 0.25*xd(2) + 0.40141790192146054*theta_dot(0) + -0.064423930130103196*c(0) + -0.074667439470176511*s(0)
Lambda solution status: 1
V solution status: 1


$ V(x)=(0.011618108 - 0.011609989s_{0} - 0.011609929c_{0} - 0.000000271theta_dot_{0} + 0.005800105s_{0} c_{0} + 0.000000116s_{0} theta_dot_{0} + 0.000000097c_{0} theta_dot_{0} + 0.002900870s_{0}^{2} + 0.002900840c_{0}^{2} + 0.000005495theta_dot_{0}^{2})$

$ u(x)=(-0.597731798 - 4.830232084s_{0} + 0.064460729c_{0} - 0.301466133theta_dot_{0})$

In [115]:
# finding the bound

from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

negative_infinity = float('-inf')
coeff = 1e-2

def find_implicit_lambdas(x, xd, V, Vdot, g):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
    prog.AddIndeterminates(xd)
    rho = prog.NewContinuousVariables(1, 'rho')[0]
        
    y = np.hstack([x, xd])
    
    l_deg = math.ceil(Vdot.TotalDegree()/2) * 2
    
    lambda_ = prog.NewSosPolynomial(Variables(y), l_deg)[0]
    
    lambda_g = []
    
    for i in range(g.size): 
        gi_deg = max(l_deg + Vdot.TotalDegree() - g[i].TotalDegree(), 0)
        lambda_g.append(prog.NewFreePolynomial(Variables(y), gi_deg))
        
    lambda_g = np.array(lambda_g)    
    
    s_deg = max(l_deg + Vdot.TotalDegree() - 2, 0)
    lambda_s = prog.NewFreePolynomial(Variables(y), s_deg)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)    
    # level set as optimization variable
    prog.AddSosConstraint(Polynomial(x.dot(x))*(V - rho) -lambda_*Vdot + lambda_g.dot(g) + lambda_s*trig)
    
    prog.AddLinearCost(-rho)
        
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
                
    print("Lambda solution status: " + str(k))
    
    assert result.is_success()
    
    lambda_g_results = []
    
    for i in range(g.size):
        lambda_g_results.append(result.GetSolution(lambda_g[i]).RemoveTermsWithSmallCoefficients(coeff))
    
    return result.GetSolution(lambda_).RemoveTermsWithSmallCoefficients(coeff), np.array(lambda_g_results), \
result.GetSolution(lambda_s).RemoveTermsWithSmallCoefficients(coeff), result.GetSolution(rho)


def problem_solver_implicit(S, V_degree, G):
    prog = MathematicalProgram()
    s = prog.NewIndeterminates(1, "s")
    c = prog.NewIndeterminates(1, "c")
    theta_dot = prog.NewIndeterminates(1, "theta_dot")
    
    xd = prog.NewIndeterminates(3, 'xd')
    
    state_V = np.hstack([theta_dot, c, s, [1]])
    
    x = np.hstack([s, c, theta_dot])
    
    u = -G@x
        
    g0 = xd[0] - c[0]*theta_dot[0]
    g1 = xd[1] + s[0]*theta_dot[0]
    g2 = p.mass()*(p.length()**2)*xd[2] + p.damping()*theta_dot[0] - p.mass()*p.gravity()*p.length()*s[0] - u
    
    g = np.array([g0, g1, g2])
    
    V = state_V.T@S@state_V
    Vdot = V.Jacobian(x).dot(xd)
    
    V = Polynomial(V)
    Vdot = Polynomial(Vdot)
    
    g_poly = []
    for i in range(g.size):
        g_poly.append(Polynomial(g[i]))
        
    g_poly = np.array(g_poly)
    
    display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
    for i in range(1):
        lambda_, lambda_g, lambda_s, rho = find_implicit_lambdas(x, xd, V, Vdot, g_poly)
        
#         display(Markdown("$ \lambda(x)="+ToLatex(lambda_s.ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[0].ToExpression())+"$"))
#         display(Markdown("$ \lambda_g(x)="+ToLatex(lambda_g[1].ToExpression())+"$"))
        
        #V, Q = find_implicit_V(x, xd, lambda_, lambda_g, lambda_s, g_poly, V_degree)
        print(rho)
        
        Vdot = V.Jacobian(x).dot(xd)
     
        display(Markdown("$ V(x)="+ToLatex(V.ToExpression(), 9)+"$"))
        
    return V, Q

V_degree = 2
V, Q = problem_solver_implicit(candidate_matrix_new, V_degree, G_casted)

$ V(x)=(2.842200000 - 2.840000000c_{0} + 0.042000000s_{0} theta_dot_{0} + 0.051600000theta_dot_{0}^{2})$

Lambda solution status: 1
0.0021999878517211164


$ V(x)=(2.842200000 - 2.840000000c_{0} + 0.042000000s_{0} theta_dot_{0} + 0.051600000theta_dot_{0}^{2})$

In [199]:
var_list = list(V.indeterminates())
s = var_list[0]
c = var_list[1]
theta_dot = var_list[2]

In [200]:
# Plot the results as contour plots.
nq = 151
nqd = 151
q = np.linspace(-2 * np.pi, 2 * np.pi, nq)
qd = np.linspace(-50 * p.mass()*p.gravity()*p.length(), 50 * p.mass()*p.gravity()*p.length(), nqd)
Q, QD = np.meshgrid(q, qd)
Energy = .5 * p.mass() * p.length()**2 * QD**2 + p.mass()*p.gravity()*p.length() * (1 - np.cos(Q))
Vplot = Q.copy()
env = {s: 0., c: 1., theta_dot: 0}
for i in range(nq):
    for j in range(nqd):
        env[s] = np.sin(Q[i, j])
        env[c] = np.cos(Q[i, j])
        env[theta_dot] = QD[i, j]
        Vplot[i, j] = V.Evaluate(env)

# plt.rc("text", usetex=True)
fig, ax = plt.subplots()
ax.contour(Q, QD, Vplot)
ax.contour(Q, QD, Vplot, levels = [0.009], colors = 'r')
ax.contour(Q, QD, Energy, alpha=0.5, linestyles="dashed")
ax.set_xlabel("theta")
ax.set_ylabel("thetadot")
ax.set_title("V (solid) and Mechanical Energy (dashed)")

AttributeError: 'pydrake.examples.acrobot.AcrobotParams' object has no attribute 'mass'

In [40]:
G

array([[10.74101105,  2.42596626]])

In [225]:
prog = MathematicalProgram()
s = prog.NewIndeterminates(1, 's')
c = prog.NewIndeterminates(1, 'c')
theta = prog.NewIndeterminates(1, 'theta')

state = np.hstack([s, c, theta])

V, Q = prog.NewSosPolynomial(Variables(state), 2)

display(Markdown("$"+ToLatex(Q)+"$"))

$\begin{bmatrix} S_{0,0} & S_{1,0} & S_{2,0} & S_{3,0} \\ S_{1,0} & S_{1,1} & S_{2,1} & S_{3,1} \\ S_{2,0} & S_{2,1} & S_{2,2} & S_{3,2} \\ S_{3,0} & S_{3,1} & S_{3,2} & S_{3,3} \end{bmatrix}$

In [29]:
display(Markdown("$"+ToLatex(V.ToExpression())+"$"))

AttributeError: 'Add' object has no attribute 'ToExpression'

In [36]:
G_explicit

array([[39.4932081 ,  8.55245308]])

## Acrobot Example

In [192]:
from pydrake.examples.acrobot import (AcrobotGeometry, AcrobotInput,
                                      AcrobotPlant, AcrobotState, AcrobotParams)

In [193]:
p = AcrobotParams()

In [194]:
p.m2

<bound method PyCapsule.m2 of AcrobotParams([1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.083, 0.33, 0.1, 0.1, 9.81])>

In [195]:
plant = AcrobotPlant()
context = plant.CreateDefaultContext()
type(plant)

pydrake.examples.acrobot.AcrobotPlant

In [196]:
plant.MassMatrix(context)

array([[4.663, 2.33 ],
       [2.33 , 1.33 ]])

In [197]:
plant.DynamicsBiasTerm(context)


















array([0., 0.])

## Searching for Pendulum Control Input Explicitly

In [7]:
# explicit linearization
A_explicit = np.array([[0, 1], [p.gravity()/p.length(), -p.damping()/(p.mass()*p.length()*p.length())]])
B_explicit = np.array([[0], [1]])

Q_explicit = np.diag([10, 1])
R_explicit = [1]

X_explicit, L_explicit, G_explicit = control.care(A_explicit, B_explicit, Q_explicit, R_explicit)

In [14]:
G_explicit
G_explicit_casted = np.insert(G_explicit, 1, 0)
G_explicit_casted

array([39.4932081 ,  0.        ,  8.55245308])

In [12]:
A_cl_ex = A_explicit - B_explicit@G_explicit
Q_ly_ex = np.eye(2)
P_ex = control.lyap(A_cl_ex.T, Q_ly_ex)

In [13]:
P_ex

array([[1.39102074, 0.0251595 ],
       [0.0251595 , 0.05866096]])

In [51]:
#THIS CODE WORKS
from pydrake.solvers.mathematicalprogram import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers.csdp import CsdpSolver
from pydrake.solvers.mosek import MosekSolver

def findLambda(x, V, Vdot, count, f):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)
        
    l_deg = math.ceil(V.TotalDegree()/2) * 2
    
    lambda_, Q_l = prog.NewSosPolynomial(Variables(x), l_deg)
    
    s_deg = max(l_deg + V.TotalDegree() - 2, 0)
    
    lambda_s = prog.NewFreePolynomial(Variables(x), s_deg)
    
    u = prog.NewFreePolynomial(Variables(x), 1)
    f2 = Polynomial((-b/(m*length**2))*x[2] + (grav/length)*x[0]) + u
    f[2] = f2
    
    Vdot = V.Jacobian(x).dot(f)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)
    prog.AddSosConstraint(-Vdot - lambda_*(1 - V) + lambda_s*trig)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
    
    k = result.get_solver_details().solution_status
        
    print('Status of first solver on step ' + str(count) + ': ' + str(k))
    assert result.is_success()
    
    return result.GetSolution(lambda_), result.GetSolution(lambda_s), result.GetSolution(u)


def findV(x, lambda_, lambda_s, u, f, count):
    prog = MathematicalProgram()
    prog.AddIndeterminates(x)    
    
    V, Q = prog.NewSosPolynomial(Variables(x), 2)
            
    prog.AddLinearConstraint(V.ToExpression().Substitute({x[0]: 0, x[1]:1, x[2]: 0}) == 0)
    
    f2 = Polynomial((-b/(m*length**2))*x[2] + (grav/length)*x[0]) + u
    f[2] = f2
    
    Vdot = V.Jacobian(x).dot(f)
    
    trig = Polynomial(x[0]**2 + x[1]**2 - 1)
    prog.AddSosConstraint(-Vdot - lambda_*(1 - V) + lambda_s*trig)
    
    prog.AddLinearCost(np.trace(Q))
    
#     A = Q[0, 0]
#     B = 2*Q[1, 0]
#     C = Q[1, 1]
    
#     prog.AddCost(A)
#     prog.AddCost(C)
#     prog.AddCost(-B)
    
    solver = MosekSolver()
    result = solver.Solve(prog)
        
    k = result.get_solver_details().solution_status
                
    print('Status of second solver on step ' + str(count) + ': ' + str(k))
    assert result.is_success()
    
    return result.GetSolution(V), result.GetSolution(Q)

def setup_solver(P, G):  
    count = 0
    prog = MathematicalProgram()
    s = prog.NewIndeterminates(1, "s")
    c = prog.NewIndeterminates(1, "c")
    theta_dot = prog.NewIndeterminates(1, "\dot{O}")
    
    x = np.hstack([s, c, theta_dot])
    
    f0 = Polynomial(x[1] * x[2])
    f1 = Polynomial(-x[0] * x[2])
    f2 = Polynomial((-b/(m*length**2))*x[2] + (grav/length)*x[0] - G@x)
    
    f = np.array([f0, f1, f2])
    
    V = 2.78 - 2.78*c[0] + 0.0586*theta_dot[0]**2 + 0.053*s[0]*theta_dot[0]
    
    V = Polynomial(V)
    
    display(Markdown("$V(x) = " + ToLatex(V.RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
    Vdot = V.Jacobian(x).dot(f)
    
    for i in range(1): 
        lambda_, lambda_s, u = findLambda(x, V, Vdot, count, f)
        
#         display(Markdown("$\lambda (x) = " + ToLatex(
#         Polynomial(lambda_).RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
        V, Q = findV(x, lambda_, lambda_s, u, f, count)
        
        display(Markdown("$V(x) = " + ToLatex(V.RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        display(Markdown("$u(x) = " + ToLatex(u.RemoveTermsWithSmallCoefficients(1e-5).ToExpression(),6) + "$"))
        
        Vdot = V.Jacobian(x).dot(f)
        
        count += 1
    return V, Q
    
    
V, Q = setup_solver(P_ex, G_explicit_casted)

$V(x) = (2.780000 - 2.780000c_{0} + 0.053000s_{0} \dot{O}_{0} + 0.058600\dot{O}_{0}^{2})$

Status of first solver on step 0: 1
Status of second solver on step 0: 7


AssertionError: 